# Exploring dense convolution

This notebook uses the fiber-tree emulator to display the behaviour of various 1-D convolutions for dense data. Because the data is assumed to be dense we use the position-based operators on the premise that for dense data the position and coordinate are the same.

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=uncompressed --animation=spacetime

## Convolution Inputs Selection

Using sliders to select the shapes of the weights and input activations

In [ ]:
# Initial values

S = 3
W = 8
Q = W-S+1

density = [1.0]
seed = 10

enable_log = False

def set_params(rank_S, rank_W, rand_seed, log):
    global S
    global W
    global Q
    global density
    global seed
    global enable_log
    
    S = rank_S
    W = rank_W
    Q = W-S+1
    
    seed = rand_seed

    enable_log = (log == 'enable')

def log(*args):
    if enable_log:
        print(*args)

controls = interactive(set_params,
                       rank_S=widgets.IntSlider(min=2, max=9, step=1, value=S),
                       rank_W=widgets.IntSlider(min=2, max=12, step=1, value=W),
                       rand_seed=widgets.IntSlider(min=0, max=100, step=1, value=seed),
                       log=['disable', 'enable'])


display(controls)
createRunallButton()

## Create Input Tensors

Given shapes selected above create and display the filter weights (**f**) and input activations (**i**) and a reference output (**o_verify**)

In [ ]:
i_raw = [random.randint(1, 9) for i in range(W)]
f_raw = [random.randint(1, 9) for i in range(S)]
o_verify_raw = [0 for i in range(Q)]

for q in range(Q):
    for s in range(S):
        w = q+s
        o_verify_raw[q] += i_raw[w] * f_raw[s]

i = Tensor.fromUncompressed(["W"], i_raw)
f = Tensor.fromUncompressed(["S"], f_raw)
o_verify = Tensor.fromUncompressed(["Q"], o_verify_raw)

i.setName("I").setColor("blue")
f.setName("F").setColor("green")
o_verify.setName("O")

print("Input activations")
displayTensor(i)
print("Filter Weights")
displayTensor(f)
print("Output activations (expected)")
displayTensor(o_verify)


## Output Stationary

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

# Hack to fill in all the entries in o
o_q = o.getRoot()
o_q << Fiber(coords=range(Q), initial=1)

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q in range(Q):
    log(f"Processing output: ({q}, ({o[q]}))")
    for w in range(W):
        s = w - q
        if s < 0 or s >= S: continue
        log(f"Processing input: ({w}, {i[w]})")
        log(f"  Processing filter weight ({s}, {f[s]})")
        o[q] += f[s] * i[w]

        addActivity(canvas, (), [(w,) for w in range(q, q+S)], (), worker="W")
        addFrame(canvas, (w-q,), (w,), (q,))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Weight Stationary

In [ ]:
o = Tensor(rank_ids=["Q"], name="O")

# Hack to fill in all the entries in o
o_q = o.getRoot()
o_q << Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

print("Convolution")

for s in range(S):
    log(f"Processing weight: ({s}, {f[s]})")
    for q in range(Q):
        w = q+s
        log(f"  Processing input ({w}, {i[w]}")
        log(f"  Processing output ({q}, {o[q]})")
        
        o[q] += f[s] * i[w]
        
        addActivity(canvas, (), [(w,) for w in range(q, q+S)], (), worker="W")
        addFrame(canvas, (s,), (q+s,), (q,))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Input Stationary

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

# Hack to fill in all the entries in o
o_q = o.getRoot()
o_q << Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

print("Convolution")

for w in range(W):
    log(f"Processing input: ({w}, {i[w]})")
    for q in range(Q):
        s = w - q
        if s < 0 or s >= S: continue
        log(f"  Processing filter weight ({s}, {f[s]})")
        log(f"  Processing output ({q}, {o[q]}")
        o[q] += f[s] * i[w]

        addActivity(canvas, (),
                            [(w,) for w in range(q, q+S)], 
                            [(q,) for q in range(max(0,w-S+1),min(Q,w+1))],
                            worker="W")
        addFrame(canvas, (w-q,), (w,), (q,))


displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Output Stationary - Parallel Weight Processing

Assumes parallelism equal to the number of weights

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

# Hack to fill in all the entries in o
o_q = o.getRoot()
o_q << Fiber(coords=range(Q), initial=1)
print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q in range(Q):
    log(f"Processing output: ({q}, {o[q]})")
    for w in range(W):
        s = w - q
        if s < 0 or s >= S: continue
        log(f"Processing input: ({w}, {i[w]})")
        log(f"  Processing filter weight ({s}, {f[s]})")
        o[q] += f[s] * i[w]

        addActivity(canvas, (s,), (w,), (q,), worker=f"PE{s}")
#    addActivity(canvas, (), [(w,) for w in range(q, q+S)], (), worker="W")
    addFrame(canvas)

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Testing area

For running alternative algorithms